In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.5 MB/s eta 0:00:00


In [45]:
# app.py
%%writefile app.py

# app.py
"""
Streamlit app (updated loader) for loading a saved Iris model and:
- Prediction UI (inputs -> prediction + probabilities)
- Data exploration (histograms, scatter plots)
- Robust model-loading from dict/tuple / GridSearchCV etc.
"""

import streamlit as st
import pandas as pd
import numpy as np
from joblib import load
from sklearn import datasets
import matplotlib.pyplot as plt
import types
import traceback

st.set_page_config(page_title="Iris Classifier: Predict & Explore", layout="wide")

# ---------------------------
# Robust model extraction
# ---------------------------
def _recursive_find_model(obj):
    """Search an object (possibly nested dict/list) for a sklearn-like estimator (has .predict).
       Returns tuple (model_obj, class_names, path_str) or (None, None, None).
    """
    visited = set()
    def _search(o, path="root"):
        oid = id(o)
        if oid in visited:
            return None
        visited.add(oid)

        # direct estimator
        if hasattr(o, "predict") and callable(getattr(o, "predict")):
            return o, None, path

        # GridSearchCV / RandomizedSearchCV-like wrapper
        if hasattr(o, "best_estimator_") and hasattr(o.best_estimator_, "predict"):
            return o.best_estimator_, None, path + "['best_estimator_']"

        # If it's a dict, look for common keys first and then search nested values
        if isinstance(o, dict):
            # quick-check for common keys
            for key in ("model", "estimator", "pipeline", "clf", "classifier", "trained_model", "best_estimator", "est"):
                if key in o:
                    candidate = o[key]
                    if hasattr(candidate, "predict"):
                        # try to find class names in same dict
                        class_names = None
                        for cn_key in ("class_names", "target_names", "labels", "classes"):
                            if cn_key in o:
                                class_names = o[cn_key]
                                break
                        return candidate, class_names, path + f"['{key}']"
            # otherwise deep search values
            for k, v in o.items():
                res = _search(v, path + f"['{k}']")
                if res:
                    return res

        # list / tuple
        if isinstance(o, (list, tuple)):
            for idx, item in enumerate(o):
                res = _search(item, path + f"[{idx}]")
                if res:
                    return res

        # object attributes (instances)
        if hasattr(o, "__dict__"):
            for k, v in vars(o).items():
                res = _search(v, path + f".{k}")
                if res:
                    return res

        return None

    result = _search(obj)
    if result:
        # result is (model_obj, class_names, path)
        return result
    return None, None, None

@st.cache_resource
def load_model(path: str):
    """Load file and attempt to extract a model and class names."""
    try:
        raw = load(path)
    except Exception as e:
        return None, None, {"error": f"joblib.load failed: {e}", "trace": traceback.format_exc()}

    # If raw itself is a model-like object
    if hasattr(raw, "predict") and callable(getattr(raw, "predict")):
        return raw, None, {"type": type(raw).__name__, "detail": "top-level estimator"}

    # If it's a GridSearchCV-like object saved directly
    if hasattr(raw, "best_estimator_") and hasattr(raw.best_estimator_, "predict"):
        return raw.best_estimator_, None, {"type": type(raw).__name__, "detail": "best_estimator_ extracted"}

    # If it's a dict/list/tuple, try to find inner estimator
    model_obj, class_names, path = _recursive_find_model(raw)
    if model_obj is not None:
        meta = {"type": type(model_obj).__name__, "found_at": path}
        # try to also expose raw keys if top-level dict
        if isinstance(raw, dict):
            meta["raw_keys"] = list(raw.keys())
        return model_obj, class_names, meta

    # fallback: no model found
    meta = {"type": type(raw).__name__, "raw_preview": None}
    if isinstance(raw, dict):
        meta["raw_keys"] = list(raw.keys())
    if isinstance(raw, (list, tuple)):
        meta["len"] = len(raw)
    # small preview
    try:
        meta["raw_preview"] = str(raw)[:500]
    except Exception:
        meta["raw_preview"] = "<unable to preview raw object>"
    return None, None, meta

# ---------------------------
# Other helpers
# ---------------------------
@st.cache_resource
def load_sample_data():
    iris = datasets.load_iris(as_frame=True)
    df = iris.frame.copy()
    df = df.rename(columns={
        'sepal length (cm)': 'sepal_length',
        'sepal width (cm)': 'sepal_width',
        'petal length (cm)': 'petal_length',
        'petal width (cm)': 'petal_width',
        'target': 'target'
    })
    target_names = iris.target_names.tolist()
    df['target_name'] = df['target'].map(lambda t: target_names[int(t)])
    return df, list(df.columns[:4]), target_names

def predict_and_format(model, X: np.ndarray, provided_class_names=None):
    if model is None:
        return None, (None, None)
    preds = model.predict(X)
    probs = None
    try:
        probs = model.predict_proba(X)
    except Exception:
        probs = None

    # Determine class labels order for probabilities
    class_labels = None
    if provided_class_names:
        class_labels = list(provided_class_names)
    else:
        try:
            if hasattr(model, "classes_"):
                class_labels = list(model.classes_)
        except Exception:
            class_labels = None

    return preds, (probs, class_labels)

# ---------------------------
# App layout & behavior
# ---------------------------
st.title("Iris Species Classifier — Predict & Explore (robust loader)")
st.markdown("This app attempts to find a species of flower based on its measurements.")

MODEL_PATH = "/content/drive/MyDrive/Major_Project_1/iris_model.joblib"
model, saved_class_names, model_meta = load_model(MODEL_PATH)
sample_df, feature_names, default_target_names = load_sample_data()
target_names = saved_class_names if saved_class_names is not None else default_target_names

# Sidebar
with st.sidebar:
    st.header("Controls")
    mode = st.radio("Mode", ["Prediction", "Data Exploration"], index=0)
    st.markdown("---")
    st.write("Model status:")
    if model is None:
        st.error("No usable model found.")
        # show helpful debugging info
        st.json(model_meta)
        st.info("If you saved a dict like {'model': estimator, 'class_names': [...]}, this loader will find it. If not found, please share the joblib keys or re-save your estimator as the top-level object.")
    else:
        st.success("Model loaded.")
        st.caption(f"Extracted model type: `{model_meta.get('type','?')}`")
        if "found_at" in model_meta:
            st.caption(f"Found at: {model_meta['found_at']}")
        if model_meta.get("raw_keys"):
            st.write("Top-level keys:", ", ".join(map(str, model_meta["raw_keys"])))
    st.markdown("---")
    st.write(f"Sample dataset: {sample_df.shape[0]} rows • features: {', '.join(feature_names)}")

# Prediction Mode
if mode == "Prediction":
    st.subheader("Prediction Mode")
    st.markdown("Enter measurements and click **Predict**.")

    left_col, right_col = st.columns([1, 1])
    with left_col:
        def slider_for_feature(df, feat):
            fmin = float(df[feat].min())
            fmax = float(df[feat].max())
            frange = fmax - fmin
            return st.slider(
                label=feat.replace("_", " ").title(),
                min_value=round(fmin - 0.2 * frange, 2),
                max_value=round(fmax + 0.2 * frange, 2),
                value=float(df[feat].median()),
                step=0.01,
                help=f"Measured {feat.replace('_',' ')} in cm (approx: {fmin:.2f}–{fmax:.2f})"
            )

        sepal_length = slider_for_feature(sample_df, "sepal_length")
        sepal_width  = slider_for_feature(sample_df, "sepal_width")
        petal_length = slider_for_feature(sample_df, "petal_length")
        petal_width  = slider_for_feature(sample_df, "petal_width")

        st.markdown("#### Batch input (optional)")
        batch_text = st.text_area(
            "Paste CSV rows (no header) to predict multiple at once (order: sepal_length,sepal_width,petal_length,petal_width)",
            value="",
            height=80
        )
        predict_button = st.button("Predict")

    with right_col:
        st.markdown("### Prediction result")
        if predict_button:
            # parse input
            try:
                if batch_text.strip():
                    rows = [r.strip() for r in batch_text.strip().splitlines() if r.strip()]
                    parsed = []
                    for r in rows:
                        parts = [p.strip() for p in r.split(",")]
                        if len(parts) != 4:
                            raise ValueError("Each row must have 4 comma-separated values.")
                        parsed.append([float(x) for x in parts])
                    X = np.array(parsed)
                else:
                    X = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
            except Exception as e:
                st.error(f"Input parsing error: {e}")
                X = None

            if X is not None:
                if model is None:
                    st.error("Model not available — cannot predict. See sidebar for diagnostics.")
                else:
                    try:
                        preds, (probs, class_labels) = predict_and_format(model, X, provided_class_names=saved_class_names)
                        for i, single_pred in enumerate(preds):
                            # determine display name
                            pred_name = None
                            try:
                                # if model returns numeric labels and default target names exist
                                if isinstance(single_pred, (int, np.integer)):
                                    pred_name = target_names[int(single_pred)]
                                else:
                                    pred_name = str(single_pred)
                            except Exception:
                                pred_name = str(single_pred)

                            # simple color coding
                            color_map = {
                                target_names[0]: "#138000",
                                target_names[1]: "#e67700",
                                target_names[2]: "#c50000",
                            }
                            display_color = color_map.get(pred_name, "#333333")
                            st.markdown(f"**Sample {i+1} prediction:**")
                            st.markdown(f"<div style='padding:10px;border-radius:8px;background:{display_color};color:white;'><strong style='font-size:18px'>{pred_name}</strong></div>", unsafe_allow_html=True)

                            if probs is not None and class_labels is not None:
                                prob_series = pd.Series(probs[i], index=class_labels)
                                st.markdown("Prediction probabilities:")
                                st.bar_chart(prob_series)
                            elif probs is not None:
                                # if we don't know class labels, display numeric columns
                                prob_df = pd.DataFrame(probs, columns=[f"c{i}" for i in range(probs.shape[1])])
                                st.markdown("Prediction probabilities (no class labels available):")
                                st.table(prob_df.iloc[i])
                    except Exception as e:
                        st.error(f"Prediction error: {e}")
                        st.exception(traceback.format_exc())
        else:
            st.info("Set inputs and click Predict.")

    st.markdown("---")
    if st.checkbox("Show current input as table"):
        input_df = pd.DataFrame([{
            "sepal_length": sepal_length,
            "sepal_width": sepal_width,
            "petal_length": petal_length,
            "petal_width": petal_width
        }])
        st.table(input_df)

# Data Exploration Mode
else:
    st.subheader("Data Exploration Mode")
    st.markdown("Explore the sample Iris dataset.")

    exp_col1, exp_col2 = st.columns([1, 2])
    with exp_col1:
        st.markdown("### Histogram")
        hist_feature = st.selectbox("Choose feature for histogram", options=feature_names, index=0)
        bins = st.slider("Bins", min_value=5, max_value=50, value=15)

        st.markdown("### Scatter plot")
        x_feat = st.selectbox("X axis", options=feature_names, index=0)
        y_feat = st.selectbox("Y axis", options=feature_names, index=2)
        hue_by_target = st.checkbox("Color by species", value=True)

    with exp_col2:
        fig_hist, ax_hist = plt.subplots()
        ax_hist.hist(sample_df[hist_feature], bins=bins)
        ax_hist.set_xlabel(hist_feature.replace("_", " ").title())
        ax_hist.set_ylabel("Count")
        ax_hist.set_title(f"Histogram of {hist_feature.replace('_',' ').title()}")
        st.pyplot(fig_hist)

        fig_scat, ax_scat = plt.subplots()
        if hue_by_target:
            for t_name in sample_df['target_name'].unique():
                sub = sample_df[sample_df['target_name'] == t_name]
                ax_scat.scatter(sub[x_feat], sub[y_feat], label=t_name, alpha=0.8, edgecolors='w', s=60)
            ax_scat.legend(title="Species")
        else:
            ax_scat.scatter(sample_df[x_feat], sample_df[y_feat], alpha=0.8)
        ax_scat.set_xlabel(x_feat.replace("_"," ").title())
        ax_scat.set_ylabel(y_feat.replace("_"," ").title())
        ax_scat.set_title(f"{y_feat.replace('_',' ').title()} vs {x_feat.replace('_',' ').title()}")
        st.pyplot(fig_scat)

    st.markdown("---")
    if st.checkbox("Show raw dataset table"):
        st.dataframe(sample_df)

# Footer
st.markdown("---")
st.markdown(
    """
    **Notes**
    - THIS MODEL WILL PREDICT DIFFERENT SPECIES OF FLOWERS   """
)


Overwriting app.py


In [42]:
!ngrok config add-authtoken 30ACubmdi0jxHkaCPKpc9Dg9um8_6tPcJmskLiruGphPRmX8v

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [43]:
import os
import threading
def run_streamlit():
  os.system('streamlit run app.py --server.port 8501')
thread=threading.Thread(target=run_streamlit)
thread.start()

In [44]:
from pyngrok import ngrok
import time

# Disconnect any existing tunnels to avoid the limit
for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)

time.sleep(5)
public_url = ngrok.connect(8501)
print("your streamlit app is live here:", public_url)

your streamlit app is live here: NgrokTunnel: "https://32ba8b3af5d6.ngrok-free.app" -> "http://localhost:8501"
